In [9]:
GYRO = False
SEQUENCE_LENGTH = 4
SEQUENCE_OVERLAP = 3
BATCH_SIZE = 10
EPOCHS = 10
CATEGORY = "moving" # "moving" or "stationary"
MODEL_NAME = f"category:{CATEGORY}_epochs:{EPOCHS}_batch:{BATCH_SIZE}_gyro:{GYRO}_window:{SEQUENCE_LENGTH}_overlap:{SEQUENCE_OVERLAP}"
DEV_SIZE = 7
TEST_SIZE = 7
LEAVE_ONE_OUT = False

### This code takes in the type of activity, ie "stationary" or "moving" and trains a model for just classifying which phsyical activity of that category the data is  

In [10]:
import file_tagger
import sequence_genrator
import tensorflow as tf
import numpy as np
from keras import layers, Sequential, models
from sklearn.model_selection import train_test_split
import split_by_student


STATIONARY_ACTIVITIES = {
    "sitting",
    "standing",
    "lying_down_back",
    "lying_down_stomach",
    "lying_down_right",
    "lying_down_left"
}

STATIONARY_ACTIVITIES_COMBINED = {
    "sitting_or_standing",
    "lying_down_back",
    "lying_down_stomach",
    "lying_down_right",
    "lying_down_left"
}

MOVING_ACTIVITIES = {
    "walking",
    "ascending_stairs",
    "descending_stairs",
    "shuffle_walking",
    "running",
    "misc_movements"
}


DATA_DIRECTORY = "./all_respeck"

activity_dict = {"moving": MOVING_ACTIVITIES,
                       "stationary": STATIONARY_ACTIVITIES
            }

POSSIBLE_ACTIVITIES = activity_dict[CATEGORY]

if CATEGORY == "stationary":
    POSSIBLE_OUTCOMES = STATIONARY_ACTIVITIES_COMBINED
else:
    POSSIBLE_OUTCOMES = POSSIBLE_ACTIVITIES

LABEL_TO_INDEX = {label: idx for idx, label in enumerate(POSSIBLE_OUTCOMES)}


In [11]:
def generate_training_data(directory, sequence_length, overlap, file_names, gyro = GYRO): # if gyro is false, only accelerometer data is used

    tagged_data = []

    # group each csv file into their respective areas
    csv_dictionary = file_tagger.tag_directory(directory)

    # iterates through each activity
    for key in csv_dictionary.keys():
        
        
        activity = key.split("&")[0]
        if activity in POSSIBLE_ACTIVITIES:

            # iterates through each csv file for the activity 
            for csv_file in csv_dictionary[key]:
                if csv_file in file_names:
                    if gyro:
                        sequences = sequence_genrator.generate_sequences_from_file_with_gyroscope(directory + "/" + csv_file, sequence_length, overlap)
                    else:
                        sequences = sequence_genrator.generate_sequences_from_file_without_gyroscope(directory + "/" + csv_file, sequence_length, overlap)

                    # iterate through each generated sequence
                    for sequence in sequences:
                        if activity == "standing" or activity == "sitting":
                            tagged_data.append(("sitting_or_standing", sequence))
                        else:
                            tagged_data.append((activity, sequence))

    print ("there are " + str(len(tagged_data)) + " tagged sequences in the dataset")
    return tagged_data

In [12]:
def train_model_CNN(input_data, labels_encoded, unique_labels, epochs, batch_size, validation_data):
    if GYRO:
        width = 6
    else:
        width = 3
    # Define the CNN model for your specific input shape
    model = Sequential([
        layers.Conv1D(32, 3, activation='relu', input_shape=(SEQUENCE_LENGTH*25, width)),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(128, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Dropout(0.5),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(len(unique_labels), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    if len(validation_data[0]) == 0:
        model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size)
    else:
        model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    return model

In [13]:
def create_sequence_label_lists(tagged_sequences):
    sequences = [sequence for _, sequence in tagged_sequences]
    labels = [label for label, _ in tagged_sequences]
    sequences = np.array(sequences, dtype=np.float32)
    labels_encoded = [LABEL_TO_INDEX[label] for label in labels]
    labels = np.array(labels_encoded)

    return sequences, labels
    

def create_data_sets(dev_size, test_size):

    training_files, dev_files, test_files = split_by_student.split_data(students_in_dev_set= dev_size, students_in_test_set=test_size)

    tagged_training_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=training_files)
    tagged_dev_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=dev_files)
    tagged_test_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=test_files)

    train_data, train_labels = create_sequence_label_lists(tagged_training_sequences)
    dev_data, dev_labels = create_sequence_label_lists(tagged_dev_sequences)
    test_data, test_labels = create_sequence_label_lists(tagged_test_sequences)

    #print(len(train_data), len(train_labels), len(dev_data), len(dev_labels), len(test_data), len(test_labels))

    return train_data, train_labels, dev_data, dev_labels, test_data, test_labels

In [ ]:
def leave_one_out():
    students = split_by_student.get_list_of_stutents()
    test_accuracies = []
    for test_student in students:
        print("testing student: " + str(test_student))
        print("training students: " + str([student for student in students if student != test_student]))
        
        test_files, training_files = split_by_student.get_list_of_files(test_student)

        tagged_training_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=training_files)
        tagged_test_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=test_files)

        train_data, train_labels = create_sequence_label_lists(tagged_training_sequences)
        test_data, test_labels = create_sequence_label_lists(tagged_test_sequences)

        model = train_model_CNN(train_data, train_labels, POSSIBLE_OUTCOMES, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=([], [])) #batch_size, epochs
        test_loss, test_accuracy = model.evaluate(test_data, test_labels)
        test_accuracies.append(test_accuracy)
        print("for student " + str(test_student) + " the accuracy is " + str(test_accuracy))
        print("average accuracy so far: " + str(sum(test_accuracies)/len(test_accuracies)))

        
    print("Accuracy for each student:")
    print(", ".join([f"{student}: {accuracy}" for student, accuracy in zip(students, test_accuracies)]))
    print("Average overall accuracy:", sum(test_accuracies)/len(test_accuracies))


In [14]:
if LEAVE_ONE_OUT:
    leave_one_out()
    exit()  

train_data, train_labels, dev_data, dev_labels, test_data, test_labels = create_data_sets(dev_size=DEV_SIZE, test_size=TEST_SIZE)


# train and save model (CHOOSE BETWEEN CNN AND LSTM)
model = train_model_CNN(train_data, train_labels, POSSIBLE_OUTCOMES, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(dev_data, dev_labels)) #batch_size, epochs


# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Save the trained model
#model.save(f"models/models_for_presentation/moving_classifier.keras")

Train Set: s63, s32, s7, s80, s36, s65, s39, s5, s44, s61, s9, s13, s96, s52, s21, s11, s66, s18, s17, s30, s40, s67, s8, s75, s57, s97, s51, s38, s74, s98, s88, s16, s46, s64, s27, s15, s60, s100, s55, s29, s71, s56, s50, s72, s86, s82, s22, s95, s87, s83, s77, s1, s93, s35, s34, s33, s23
Dev Set: s43, s92, s84, s59, s54, s91, s48
Test Set: s70, s45, s102, s12, s79, s3, s42
there are 9128 tagged sequences in the dataset
there are 999 tagged sequences in the dataset
there are 1127 tagged sequences in the dataset
Epoch 1/10
913/913 [==============================] - 3s 3ms/step - loss: 0.8619 - accuracy: 0.6572 - val_loss: 1.0583 - val_accuracy: 0.7317
Epoch 2/10
913/913 [==============================] - 3s 3ms/step - loss: 0.3726 - accuracy: 0.8712 - val_loss: 1.3268 - val_accuracy: 0.7417
Epoch 3/10
913/913 [==============================] - 3s 3ms/step - loss: 0.2557 - accuracy: 0.9113 - val_loss: 1.2882 - val_accuracy: 0.7708
Epoch 4/10
913/913 [==============================] - 3s